In [ ]:
from neo4j import GraphDatabase
import csv

# 连接到 Neo4j 数据库
uri = ""
username = ""
password = ""

driver = GraphDatabase.driver(uri, auth=(username, password))

# 加载数据到 Neo4j 数据库
def create_nodes_from_csv(tx, filename, name_index):
    with open(filename, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        headers = next(reader)
        for row in reader:
            node_properties = {headers[i]: row[i] for i in range(len(headers))}
            node_name = row[name_index]
            tx.run("CREATE (n:Patent {name: $name}) SET n = $props", name=node_name, props=node_properties)
def create_relationships_from_csv(tx, filename):
    with open(filename, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            relation_type = row[2]  # 假设第三列存储的是关系类型
            # 使用 APOC 创建动态关系
            tx.run("""
                MATCH (start:Enterprise {name: $start_name})
                MATCH (end:Patent {name: $end_name})
                CALL apoc.create.relationship(start, $relation_type, {}, end) YIELD rel
                RETURN rel
            """, parameters={'start_name': row[0], 'end_name': row[1], 'relation_type': relation_type})

with driver.session() as session:
    session.write_transaction(create_nodes_from_csv, "D:\AISocialtechnique\数据处理\专利数据.csv",1)
    session.write_transaction(create_relationships_from_csv, "D:\AISocialtechnique\数据处理\relation.csv")
    

